In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import sys
sys.path.append('..')

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split

import multiprocessing

from data_augmentaion.data_augmentator import DataAugmentor 

import json

import time

### Define research parameters

In [3]:
regression_task = False
# continuous_features = []
metric = 'accuracy'
test_size_proportion = 0.2
augment_sample = 0.5

search_pipelines = pipelines.get_adult_pipelines()
search_parameters = models.parameters

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]

In [4]:
from sklearn.datasets import make_classification

# Define the parameters of the synthetic dataset
n_samples = 600  # Total number of samples
n_features = 12   # Number of features
n_classes = 3    # Number of classes
class_weights = [0.5, 0.2, 0.3]  # Class imbalance ratio

# Create the imbalanced dataset
X, y = make_classification(n_samples=n_samples, 
                           n_features=n_features,
                           n_informative=3,
                           n_classes=n_classes,
                           n_clusters_per_class=2,
                           weights=class_weights,
                           random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
y = pd.Series(y, name="target")

In [5]:
continuous_features = X.columns.tolist()

#### split data

In [6]:
# all data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

In [7]:
display(y_train.value_counts())

0    250
2    142
1     88
Name: target, dtype: int64

In [8]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [9]:
print(f'whole adult dataset {metric} scores:')
_, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index', columns=[f'full data {metric} score'])
display(whole_dataset_result_df.T)

whole adult dataset accuracy scores:


,lg,rf,xgb
full data accuracy score,0.683333,0.683333,0.775


In [10]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df], axis=1)

### running experiments

In [11]:
best_estimators = {}
best_scores = {}
for i, s in enumerate(settings):
    start = time.time()
    print(f'{i} / {len(settings)}, {s}', end=' ')
    augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             cf_scoring = metric,
                             kw_args=s.get('kw_args', {})
                             )

    X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
    best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
    utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
                            search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
                                                orient='index',
                                                columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])
    full_results_df = pd.concat([full_results_df, result_df_balanced], axis=1)

    X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
    best_estimators[f'{i}'], best_scores[f'{i}'] = \
        utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
                                       orient='index',
                                       columns=[f'{json.dumps((list(s.values())))} {metric} score'])
    
    full_results_df = pd.concat([full_results_df, result_df], axis=1)
    print(f'{time.time() - start} seconds for settings {i}')


0 / 9, {'method': 'random'} 6.5230607986450195 seconds for settings 0
1 / 9, {'method': 'smote'} 6.824526071548462 seconds for settings 1
2 / 9, {'method': 'cf_random'} 0/108


100%|██████████| 1/1 [00:00<00:00,  7.63it/s]


50/108


100%|██████████| 1/1 [00:00<00:00,  7.68it/s]


100/108


...

150/240


100%|██████████| 1/1 [00:00<00:00, 10.53it/s]


200/240


100%|██████████| 1/1 [00:00<00:00, 11.40it/s]


77.46378922462463 seconds for settings 8


In [12]:
full_results_df

,full data accuracy score,"[""random""] balanced accuracy score","[""random""] accuracy score","[""smote""] balanced accuracy score","[""smote""] accuracy score","[""cf_random""] balanced accuracy score","[""cf_random""] accuracy score","[""cf_genetic"", {""proximity_weight"": 0.2, ""diversity_weight"": 5, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 0.2, ""diversity_weight"": 5, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] accuracy score","[""cf_genetic"", {""proximity_weight"": 0.2, ""diversity_weight"": 5, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 0.2, ""diversity_weight"": 5, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 2}] accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 2}] accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 2}] accuracy score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] accuracy score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 1}] accuracy score"
lg,0.683333,0.775000,0.683333,0.758333,0.758333,0.741667,0.725000,0.733333,0.633333,0.750000,0.641667,0.750,0.650000,0.750000,0.633333,0.75,0.633333,0.741667,0.658333
rf,0.683333,0.766667,0.741667,0.775000,0.775000,0.733333,0.758333,0.800000,0.741667,0.800000,0.733333,0.800,0.733333,0.800000,0.733333,0.80,0.708333,0.791667,0.733333
xgb,0.775000,0.783333,0.791667,0.808333,0.808333,0.758333,0.758333,0.791667,0.783333,0.808333,0.775000,0.775,0.758333,0.808333,0.775000,0.80,0.775000,0.808333,0.766667


In [14]:
full_results_df.to_csv(rf'../log/experiment_multiclass_{metric}_testsize{test_size_proportion}_augmentsample{augment_sample}.csv')